In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import math


# task1

In [2]:
def write_file(filename:str,df:pd.DataFrame,sampligFrequency:float,signalType:int=0,isPeriodic:int=0):
    myFile = open(filename,"w")
    myFile.write(f"{signalType}\n")
    myFile.write(f"{isPeriodic}\n")
    myFile.write(f"{sampligFrequency}\n")
        
        # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        #myFile.write(f"{row[0]} {row[1]}\n")
        myFile.write(f"{round(row[0], 6)} {round(row[1], 6)}\n")
    myFile.close()


In [3]:
def read_File(filePath: str):
    myfile = open(filePath, 'r')
    lines = [line.strip() for line in myfile.readlines()]
    myfile.close()
    
    samples = []
    for i in range(3):
        samples.append(int(lines.pop(0)))

    index_x = []
    index_y = []
    for row in lines:
        x = row.split(sep=" ")
        index_x.append(float(x[0]))
        index_y.append(float(x[1]))

    return [samples, index_x, index_y]


In [4]:
def Compare_Signals(file_name,Your_indices,Your_samples):      
    expected_indices=[]
    expected_samples=[]
    with open(file_name, 'r') as f:
        line = f.readline()
        line = f.readline()
        line = f.readline()
        line = f.readline()
        while line:
            # process line
            L=line.strip()
            if len(L.split(' '))==2:
                L=line.split(' ')
                V1=int(L[0])
                V2=float(L[1])
                expected_indices.append(V1)
                expected_samples.append(V2)
                line = f.readline()
            else:
                break
    print("Current Output Test file is: ")
    print(file_name)
    print("\n")
    print(len(expected_samples),(len(expected_indices)))
    if (len(expected_samples)!=len(Your_samples)) and (len(expected_indices)!=len(Your_indices)):
        print("Test case failed, your signal have different length from the expected one")
        return
    for i in range(len(Your_indices)):
        if(Your_indices[i]!=expected_indices[i]):
            print("Test case failed, your signal have different indicies from the expected one") 
            return
    for i in range(len(expected_samples)):
        if abs(Your_samples[i] - expected_samples[i]) < 0.01:
            continue
        else:
            print("Test case failed, your signal have different values from the expected one") 
            return
    print("Test case passed successfully")


In [5]:
def convolve_signals(indices1, samples1, indices2, samples2):

    indices1 = [int(idx) for idx in indices1]
    indices2 = [int(idx) for idx in indices2]

    output_indices = list(range(indices1[0] + indices2[0], indices1[-1] + indices2[-1] + 1))
    
    # Initialize the output samples with zero values
    output_samples = [0] * len(output_indices)
    
    # Perform the convolution
    for i, index in enumerate(output_indices):
        for j, idx1 in enumerate(indices1):
            for k, idx2 in enumerate(indices2):
                if idx1 + idx2 == index:
                    output_samples[i] += samples1[j] * samples2[k]
    
    return output_indices, output_samples

low pass filter


In [6]:
def get_window_method(StopBandAttenuation):
    #1 -> rectangular, 2-> hanning, 3-> hamming, 4-> blackman
    if StopBandAttenuation >= 0 and StopBandAttenuation <= 21 :
        return 1
    if StopBandAttenuation >= 22 and StopBandAttenuation <= 44 :
        return 2
    if StopBandAttenuation >= 45 and StopBandAttenuation <= 53 :
        return 3
    if StopBandAttenuation >= 54 and StopBandAttenuation <= 74 :
        return 4

print(get_window_method(50))

3


In [7]:
def get_N(fs,delta_f,window_method_num):
    normal_delta_f = float(delta_f/fs)
    N = 0
    
    if window_method_num == 1:#rectangular
        N = 0.9/normal_delta_f
    elif window_method_num == 2:#hanning
        N = 3.1/normal_delta_f
    elif window_method_num == 3:#hamming
        N = 3.3/normal_delta_f
    elif window_method_num == 4:#blackman
        N = 5.5/normal_delta_f
    
    if(int(N) % 2 == 0):
        N=int(N) + 1
    else:
        N=int(N)
    
    return N

print(get_N(1000,50,4))


111


In [8]:
def get_window_value(N,n,window_method_num):
    
    result = 0.0

    if window_method_num == 1:#rectangular
        result = 1.0    
    elif window_method_num == 2:#hanning
        rad = math.radians(float((2*180*n)/N))
        result = 0.5 + (0.5 * math.cos(rad))
    elif window_method_num == 3:#hamming
        rad = math.radians(float(2 * 180 * n/N))
        result = 0.54 + (0.46 * math.cos(rad))
    elif window_method_num == 4:#blackman
        rad1 = math.radians(float(2 * 180 * n / (N-1)))
        rad2 = math.radians(4 * 180 * n / (N-1))
        result = 0.42 + (0.5 * math.cos(rad1)) + (0.08 * math.cos(rad2))
                    
    return result    
        
print(get_window_value(53,0,3))

1.0


In [9]:
def semetric_list(lst:list):
    l = lst[:0:-1]
    #print(l)
    l.extend(lst)
    #print(l)
    return l
semetric_list([1,2,3,4])

[4, 3, 2, 1, 2, 3, 4]

In [10]:
def lowPass_filter(fs, fp, delta_f,StopBandAttenuation):
    impulse_values = []
    #find window method
    method_num = get_window_method(StopBandAttenuation)
    #print(method_num)
    
    #find N
    N = get_N(fs ,delta_f,method_num)
    max_iteration = int(N/2)

    #find norm_fc
    fc1 = fp + (delta_f/2)
    norm_fc = float(fc1/fs)
    
    #print(N,max_iteration)
    
    Hd_value = 2 * norm_fc
    Wn_value = get_window_value(N,0,method_num)
    impulse_values.append(Hd_value * Wn_value)
    #print(Hd_value * Wn_value)
    
    for n in range(1,max_iteration+1):
        rad = math.radians(n*2*180*norm_fc) 
        Hd_value = 2 * norm_fc * math.sin(rad) /(n * 2*math.pi*norm_fc)
        Wn_value = get_window_value(N,n,method_num)
        #print(Hd_value,Wn_value)
        impulse_values.append(Hd_value*Wn_value)
        #print(n)
        #print(Hd_value * Wn_value)
    impulse_values=semetric_list(impulse_values)
    
    index = [i for i in range(-max_iteration,max_iteration+1) ]
    #print (index)
    
    return impulse_values, index


h , input_index = lowPass_filter(8000,1500,500,50)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 1\LPFCoefficients.txt",input_index,h)
    

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 1\LPFCoefficients.txt


53 53
Test case passed successfully


In [11]:
readed_signal = read_File(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 2\ecg400.txt")
print(readed_signal[1])

out_ind , out_sap= convolve_signals(readed_signal[1],readed_signal[2],input_index,h)
Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 2\ecg_low_pass_filtered.txt",out_ind,out_sap)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

high pass filter


In [12]:
def highPass_filter(fs, fp, delta_f,StopBandAttenuation):
    impulse_values = []
    #find window method
    method_num = get_window_method(StopBandAttenuation)
    #print(method_num)
    
    #find N
    N = get_N(fs ,delta_f,method_num)
    max_iteration = int(N/2)

    #find norm_fc
    fc1 = fp - (delta_f/2)
    norm_fc = float(fc1/fs)
    
    #print(N,max_iteration)
    
    Hd_value = 1 - 2 * norm_fc
    Wn_value = get_window_value(N,0,method_num)
    impulse_values.append(Hd_value * Wn_value)
    #print(Hd_value * Wn_value)
    
    for n in range(1,max_iteration+1):
        rad = math.radians(n*2*180*norm_fc) 
        Hd_value = -2 * norm_fc * math.sin(rad) /(n * 2*math.pi*norm_fc)
        Wn_value = get_window_value(N,n,method_num)
        impulse_values.append(Hd_value*Wn_value)
        
    impulse_values=semetric_list(impulse_values)
    
    index = [i for i in range(-max_iteration,max_iteration+1) ]
    
    
    return impulse_values, index

h , input_index =highPass_filter(8000,1500,500,70)
Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 3\HPFCoefficients.txt",input_index,h)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 3\HPFCoefficients.txt


89 89
Test case passed successfully


In [13]:
#test 
readed_signal = read_File(r"task7/FIR test cases/Testcase 4/ecg400.txt")

out_ind , out_sap = convolve_signals(readed_signal[1],readed_signal[2],input_index,h)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 4\ecg_high_pass_filtered.txt",out_ind,out_sap)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 4\ecg_high_pass_filtered.txt


488 488
Test case passed successfully


bandPass filter


In [14]:
def bandPass_filter(fs, fp1,fp2, delta_f,StopBandAttenuation):
    impulse_values = []
    #find window method
    method_num = get_window_method(StopBandAttenuation)
    #print(method_num)
    
    #find N
    N = get_N(fs ,delta_f,method_num)
    max_iteration = int(N/2)

    #find norm_fc
    fc1 = fp1 - (delta_f/2)
    norm_fc1 = float(fc1/fs)
    
    fc2 = fp2 + (delta_f/2)
    norm_fc2 = float(fc2/fs)
    
    #print(N,max_iteration)
    
    Hd_value =  2 * (norm_fc2-norm_fc1)
    Wn_value = get_window_value(N,0,method_num)
    impulse_values.append(Hd_value * Wn_value)
    #print(Hd_value * Wn_value)
    
    for n in range(1,max_iteration+1):
        rad1 = math.radians(n*2*180*norm_fc1)
        rad2 = math.radians(n*2*180*norm_fc2)

        Hd_value = (2*norm_fc2* math.sin(rad2)/(n * 2*math.pi*norm_fc2)) - (2*norm_fc1* math.sin(rad1)/(n * 2*math.pi*norm_fc1))
        Wn_value = get_window_value(N,n,method_num)
        impulse_values.append(Hd_value*Wn_value)
        
    impulse_values=semetric_list(impulse_values)
    
    index = [i for i in range(-max_iteration,max_iteration+1) ]
    
    
    return impulse_values, index

h,input_index = bandPass_filter(1000,150,250,50,60)
Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 5\BPFCoefficients.txt",input_index,h)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 5\BPFCoefficients.txt


111 111
Test case passed successfully


In [15]:
readed_signal = read_File(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 6\ecg400.txt")

out_ind,out_sap = convolve_signals(readed_signal[1],readed_signal[2],input_index,h)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 6\ecg_band_pass_filtered.txt",out_ind,out_sap)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 6\ecg_band_pass_filtered.txt


510 510
Test case passed successfully


In [16]:
def bandStop_filter(fs, fp1,fp2, delta_f,StopBandAttenuation):
    impulse_values = []
    #find window method
    method_num = get_window_method(StopBandAttenuation)
    #print(method_num)
    
    #find N
    N = get_N(fs ,delta_f,method_num)
    max_iteration = int(N/2)

    #find norm_fc
    fc1 = fp1 + (delta_f/2)
    norm_fc1 = float(fc1/fs)
    
    fc2 = fp2 - (delta_f/2)
    norm_fc2 = float(fc2/fs)
    
    #print(N,max_iteration)
    
    Hd_value = 1- 2 * (norm_fc2-norm_fc1)
    Wn_value = get_window_value(N,0,method_num)
    impulse_values.append(Hd_value * Wn_value)
    #print(Hd_value * Wn_value)
    
    for n in range(1,max_iteration+1):
        rad1 = math.radians(n*2*180*norm_fc1)
        rad2 = math.radians(n*2*180*norm_fc2)

        Hd_value = (2*norm_fc1* math.sin(rad1)/(n * 2*math.pi*norm_fc1)) - (2*norm_fc2* math.sin(rad2)/(n * 2*math.pi*norm_fc2))
        Wn_value = get_window_value(N,n,method_num)
        impulse_values.append(Hd_value*Wn_value)
        
    impulse_values=semetric_list(impulse_values)
    
    index = [i for i in range(-max_iteration,max_iteration+1) ]
    
    
    return impulse_values, index

h , input_index = bandStop_filter(1000,150,250,50,60)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 7\BSFCoefficients.txt",input_index,h)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 7\BSFCoefficients.txt


111 111
Test case passed successfully


In [17]:
readed_signal = read_File(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 8\ecg400.txt")

out_ind,out_sap = convolve_signals(readed_signal[1],readed_signal[2],input_index,h)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 8\ecg_band_stop_filtered.txt",out_ind,out_sap)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\FIR test cases\Testcase 8\ecg_band_stop_filtered.txt


510 510
Test case passed successfully


resampling downsampling


In [18]:
def resampling_down(input_index:list , input_samples_values:list,m:int,fs, fp, delta_f,StopBandAttenuation):
    resampled_sample_values = []
    resampled_indices =[]
    #apply filter
    h,lpf_index = lowPass_filter(fs,fp,delta_f,StopBandAttenuation)
    
    indices , sample_values = convolve_signals(input_index, input_samples_values,lpf_index,h)
    
    #down sample by M factor
    resampled_sample_values = sample_values[::m]

    resampled_indices = [i+indices[0] for i in range(len(resampled_sample_values))]
    
    return resampled_indices , resampled_sample_values
   
readed_signal = read_File(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 1\ecg400.txt")
indices1 , samples_values = resampling_down(readed_signal[1],readed_signal[2],2,8000,1500,500,50)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 1\Sampling_Down.txt",indices1,samples_values)


Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 1\Sampling_Down.txt


226 226
Test case passed successfully


resampling upsampling


In [19]:
def resampling_up(input_index:list , input_samples_values:list,l:int,fs, fp, delta_f,StopBandAttenuation):
    resampled_sample_values = []
    resampled_indices =[]

    #apply up sample by L factor
    for i in input_samples_values:
        resampled_sample_values.append(i)
        
        for j in range(l-1):
            resampled_sample_values.append(0)
    
    resampled_sample_values = resampled_sample_values[:len(resampled_sample_values)-l+1]
    resampled_indices = [i for i in range(len(resampled_sample_values))]

    #print(resampled_indices)
    #print(resampled_sample_values)

    #apply filter
    h,lpf_index = lowPass_filter(fs,fp,delta_f,StopBandAttenuation)
    
    resampled_indices , resampled_sample_values = convolve_signals(resampled_indices, resampled_sample_values,lpf_index,h)

    #print(resampled_indices)
    #print(resampled_sample_values)

    #print(len(resampled_indices))
    #print(len(resampled_sample_values))
    #resampled_sample_values = resampled_sample_values[:len(resampled_sample_values)-l+1]
    #resampled_indices = resampled_indices[:len(resampled_indices)-l+1]
    return resampled_indices,resampled_sample_values


indices1 , samples_values = resampling_up(readed_signal[1],readed_signal[2],3,8000,1500,500,50)
Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 2\Sampling_Up.txt",indices1,samples_values)

Current Output Test file is: 
C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 2\Sampling_Up.txt


1250 1250
Test case passed successfully


resample up and down


In [20]:
def resampling_upAndDown(input_index:list , input_samples_values:list,l:int,m:int,fs, fp, delta_f,StopBandAttenuation):
    resampled_up_indices , resampled_up_samples = resampling_up(input_index, input_samples_values, l, fs, fp, delta_f, StopBandAttenuation)
    
    print(resampled_up_indices)
    print(resampled_up_samples)
    print(len(resampled_up_indices))
    print(len(resampled_up_samples))
    
    #down sampling
    s = resampled_up_samples[::m]
    isd = [i+resampled_up_indices[0] for i in range(len(s))]
    
    print(isd)
    print(s)
    print(len(isd))
    print(len(s))
    
    return  isd, s
    
readed_signal = read_File(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 3\ecg400.txt")    
re_index,re_samples =  resampling_upAndDown(readed_signal[1],readed_signal[2],3,2,8000,1500,500,50)

Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 3\Sampling_Up_Down.txt",re_index,re_samples)
Compare_Signals(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 3\Sampling_Up_Down1.txt",re_index,re_samples)


[-26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197

In [21]:

data = pd.DataFrame({"index":re_index,
                     "samples":re_samples})

write_file(r"C:\Users\HP\Desktop\dsp\task7\sampling test cases\Testcase 3\ss",data,len(data))